In [1]:
# !pip install yahooquery
# !pip install pathlib
# !pip install ruamel-yaml
# !pip install numexpr==2.8.0
# !pip install bottleneck==1.3.4
# !pip install openpyxl==3.0.10
# !pip install xlsxwriter==3.0.3

In [1]:
import pandas as pd
import os
from yahooquery import Ticker
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

#import asyncio
#import aiohttp

In [2]:
# Specify the folder containing your Excel file/s
folder_path = './adr-list'

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read data from the Excel file into a DataFrame
        df_adr = pd.read_excel(file_path)

In [3]:
# Check how many records
len(df_adr)

2498

In [4]:
# Filter to include ADRs only
df_adr = df_adr[df_adr['Type'] == 'ADR']

In [5]:
# Check how many records remain after filtering
len(df_adr)

2177

In [6]:
# Store ADR ticker names in a list
tickers_list = df_adr['Symbol'].tolist()

In [7]:
#4a

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Specify the columns you want to retrieve
columns_to_extract = [
    'asOfDate', 'periodType', 'NetIncome', 'GrossProfit', 'PretaxIncome',
    'TotalRevenue', 'LongTermDebt', 'LongTermDebtAndCapitalLeaseObligation',
    'TotalAssets', 'CurrentAssets', 'CurrentLiabilities', 'OperatingCashFlow',
    'DilutedEPS'
]

# List of tickers
tickers = tickers_list

def fetch_data(ticker):
    try:
        # Get income statement data for the current ticker
        financial_data = Ticker(ticker).all_financial_data()
        
        # Extract the specified columns (with null values for non-existing columns)
        data_for_ticker = {column: financial_data.get(column) for column in columns_to_extract}
        
        # Add 'ticker' as a key to the dictionary
        data_for_ticker['ticker'] = ticker
        
        # Convert the dictionary to a DataFrame and return it
        return pd.DataFrame(data_for_ticker)
    except Exception as e:
        # print(f"Error fetching data for {ticker}: {str(e)}")

        return pd.DataFrame()

In [8]:
# Parallel processing
with ThreadPoolExecutor() as executor, tqdm(total=len(tickers), desc="Fetching Data") as pbar:
    # Fetch data for all tickers concurrently
    df_list = list(executor.map(lambda ticker: (pbar.update(1) or fetch_data(ticker)), tickers))

Fetching Data: 100%|███████████████████████████████████████████████████████████████| 2177/2177 [14:22<00:00,  2.52it/s]


In [10]:
# Filter out empty DataFrames
# df_list2 = [df for df in df_list if df is not None]

In [11]:
# df_list2

In [9]:
# Filter out unsuccessful fetches (False values)
df_list1 = [df for df in df_list if isinstance(df, pd.DataFrame) and not df.empty]

In [10]:
# Combine individual DataFrames into one DataFrame
df1 = pd.concat(df_list1, ignore_index=True)

C:\Users\rexeu\AppData\Local\Temp\ipykernel_14984\1365732248.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df1 = pd.concat(df_list1, ignore_index=True)


In [11]:
df1.head()

,asOfDate,periodType,NetIncome,GrossProfit,PretaxIncome,TotalRevenue,LongTermDebt,LongTermDebtAndCapitalLeaseObligation,TotalAssets,CurrentAssets,CurrentLiabilities,OperatingCashFlow,DilutedEPS,ticker
0,2020-12-31,12M,-2.709347e+09,1.076011e+09,-2.581792e+09,4.829019e+09,3.901053e+09,5.234680e+09,1.937376e+10,6.055607e+09,6.121960e+09,7.142430e+08,-26.8200,VNET
1,2021-12-31,12M,5.000980e+08,1.438030e+09,6.651740e+08,6.189801e+09,6.481966e+09,9.885772e+09,2.309504e+10,5.324123e+09,5.179995e+09,1.387922e+09,-2.1600,VNET
2,2022-12-31,12M,-7.759520e+08,1.358256e+09,-6.304550e+08,7.065232e+09,8.909115e+09,1.286204e+10,2.694840e+10,7.052276e+09,6.332085e+09,2.440214e+09,-5.2200,VNET
3,2020-03-31,12M,2.140000e+08,NaN,2.150000e+08,3.310000e+08,NaN,5.950000e+08,8.567000e+09,NaN,NaN,NaN,0.1105,TGOPY
4,2020-09-30,12M,7.090000e+08,NaN,7.090000e+08,8.220000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TGOPY


In [12]:
len(df1['ticker'].unique())

983

In [13]:
tickers_found = df1['ticker'].unique().tolist()

In [14]:
tickers_not_found = [ticker for ticker in tickers_list if ticker not in tickers_found]

In [15]:
len(tickers_not_found)

1193

In [22]:
tickers_not_found

['TRPOY',
 'LBPS',
 'WBAI',
 'JOBS',
 'ABB',
 'ACHL',
 'ACSAY',
 'ADAP',
 'ADXN',
 'ADDLY',
 'ADDHY',
 'ADEVY',
 'AMIGY',
 'ADYEY',
 'AGGNY',
 'AENZ',
 'AERZY',
 'ARRPY',
 'AGESY',
 'AGPYY',
 'ACGBY',
 'AAGIY',
 'ADERY',
 'AIRYY',
 'AIQUY',
 'EADSY',
 'AIPUY',
 'AIIXY',
 'AJSCY',
 'AKTX',
 'AKBDY',
 'AKRCY',
 'AKZOY',
 'ALFVY',
 'ALFRY',
 'ALBBY',
 'ALIZY',
 'ALMRY',
 'ALBKY',
 'ALSYY',
 'ATHE',
 'ASGSY',
 'AWCMY',
 'ACH',
 'AMDLY',
 'AMADY',
 'SUBMY',
 'AMLYY',
 'AMFPY',
 'CTXAY',
 'AMYT',
 'AMSSY',
 'AEBZY',
 'AEBMY',
 'AELIY',
 'ANPCY',
 'ANGPY',
 'AHCHY',
 'AITUY',
 'ANSLY',
 'ATVDY',
 'ATHJY',
 'ANZBY',
 'AOWPY',
 'AMKBY',
 'AMSIY',
 'MDEPY',
 'ATWNY',
 'ARZTY',
 'AHKSY',
 'ACENY',
 'ASHTY',
 'AACEY',
 'ASCCY',
 'ASMVY',
 'ARNNY',
 'ASOZY',
 'ALPMY',
 'PTAIY',
 'ASR',
 'ATAAY',
 'ACMDY',
 'ATLCY',
 'AUOTY',
 'ADPXY',
 'AZNNY',
 'JG',
 'AAYYY',
 'ATHM',
 'AVSFY',
 'DUFRY',
 'AXFOY',
 'AYAAY',
 'AZLGY',
 'AZIHY',
 'AZZRY',
 'AZUL',
 'BOLSY',
 'IBA',
 'BCCMY',
 'BKFKY',
 'BBAR',
 'BN

In [16]:
# Write the ADR data to a new Excel file
adr_file_path = './adr-list/adr_list3.xlsx'
df1.to_excel(adr_file_path, index=False)

In [9]:
#4a

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Specify the columns you want to retrieve
columns_to_extract = [
    'asOfDate', 'periodType', 'NetIncome', 'GrossProfit', 'PretaxIncome',
    'TotalRevenue', 'DilutedEPS'
]

# List of tickers
tickers = tickers_list

def fetch_data(ticker):
    try:
        # Get income statement data for the current ticker
        financial_data = Ticker(ticker).income_statement()
        
        # Extract the specified columns (with null values for non-existing columns)
        data_for_ticker = {column: financial_data.get(column) for column in columns_to_extract}
        
        # Add 'ticker' as a key to the dictionary
        data_for_ticker['ticker'] = ticker
        
        # Convert the dictionary to a DataFrame and return it
        return pd.DataFrame(data_for_ticker)
    except Exception as e:
        # print(f"Error fetching data for {ticker}: {str(e)}")

        return pd.DataFrame()

In [10]:
# Parallel processing
with ThreadPoolExecutor() as executor, tqdm(total=len(tickers), desc="Fetching Data") as pbar:
    # Fetch data for all tickers concurrently
    df_list = list(executor.map(lambda ticker: (pbar.update(1) or fetch_data(ticker)), tickers))

Fetching Data: 100%|███████████████████████████████████████████████████████████████| 2177/2177 [13:17<00:00,  2.73it/s]


In [12]:
# Filter out unsuccessful fetches (False values)
df_list1 = [df for df in df_list if isinstance(df, pd.DataFrame) and not df.empty]

In [13]:
# Combine individual DataFrames into one DataFrame
df1 = pd.concat(df_list1, ignore_index=True)

C:\Users\rexeu\AppData\Local\Temp\ipykernel_25276\1365732248.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df1 = pd.concat(df_list1, ignore_index=True)


In [14]:
len(df1['ticker'].unique())

952